In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
df = pd.read_csv('test.csv')

X = df.select_dtypes(include=[np.number]).dropna()

vif_data = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i) 
            for i in range(X.shape[1])]
})

pd.set_option('display.float_format', '{:,.2f}'.format)

vif_data


c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,feature,VIF
0,년도,"12,360.46"
1,월,9.74
2,전기차 등록수,14.44
3,전기차 순증가량,4.18
4,주행거리,248.88
5,에너지소비효율,525.91
6,충전지용량,224.20
7,급속 충전기 설치건수,10.91
8,급속 충전기 누적건수,36.47
9,완속 충전기 설치건수,14.16


In [ ]:
def calculate_vif(X):
    """
    입력된 X(DataFrame)에 대해 각 컬럼의 VIF를 계산하여 DataFrame으로 리턴
    """
    vif_df = pd.DataFrame({
        'feature': X.columns,
        'VIF': [
            variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])
        ]
    })
    return vif_df

def drop_high_vif(X, thresh=10.0, verbose=True):
    """
    VIF가 thresh보다 큰 변수가 없을 때까지,
    가장 VIF가 높은 변수를 하나씩 제거하는 반복문.
    """
    features = X.columns.tolist()
    while True:
        vif_df = calculate_vif(X[features])
        max_vif = vif_df['VIF'].max()
        if verbose:
            print(vif_df.sort_values('VIF', ascending=False).round(2))
        if max_vif <= thresh:
            if verbose:
                print(f"\n모든 VIF가 {thresh} 이하로 내려왔습니다 (최대 VIF = {max_vif:.2f})")
            break
        drop_feature = vif_df.sort_values('VIF', ascending=False)['feature'].iloc[0]
        if verbose:
            print(f"→ 제거: {drop_feature} (VIF={max_vif:.2f})\n")
        features.remove(drop_feature)
    return features, vif_df

if __name__ == "__main__":
    df = pd.read_csv('test.csv')
    df_numeric = df.select_dtypes(include=[np.number]).dropna()
    remaining_features, final_vif = drop_high_vif(df_numeric, thresh=10.0)
    print("최종 남은 변수:", remaining_features)

c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


               feature               VIF
12          차량 순증가량_총계               inf
13             수소차 등록수               inf
14            수소차 순증가량               inf
11           차량 등록수_총계               inf
27                  자산 12,259,479,898.60
29                순자산액  8,455,677,678.18
28                  부채    359,201,958.99
17            평균기온(°C)         12,569.31
0                   년도         12,360.46
23                 cpi         10,435.72
18          평균최고기온(°C)          4,773.10
19          평균최저기온(°C)          2,287.36
5              에너지소비효율            525.91
26                경상소득            279.75
4                 주행거리            248.88
6                충전지용량            224.20
25              평균기준금리             70.03
21          미세먼지2.5 수치             61.60
15          하이브리드차 등록수             55.76
22           미세먼지10 수치             39.50
8          급속 충전기 누적건수             36.47
16         하이브리드차 순증가량             30.85
10         완속 충전기 누적건수             29.45
24              

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import random

def compute_max_vif(X):
    """DataFrame X의 변수들에 대해 max VIF를 계산."""
    vifs = [
        variance_inflation_factor(X.values, i)
        for i in range(X.shape[1])
    ]
    return max(vifs)

def random_feature_search(df, n_iter=1000, thresh=10.0, random_state=None):
    """
    무작위로 변수 하나를 제거해가며 최적 조합을 찾는 단순 Random Search
    - df: VIF 계산 대상 DataFrame (수치형 변수만)
    - n_iter: 반복 횟수
    - thresh: 이상적 max VIF 임계값
    """
    if random_state is not None:
        random.seed(random_state)
        np.random.seed(random_state)

    features = list(df.columns)
    best_subset = features.copy()
    best_score = -np.inf
    best_vif  = None

    for it in range(n_iter):
        # 현재 후보 집합: best_subset 에서 랜덤하게 하나를 빼 본다
        cand = best_subset.copy()
        drop = random.choice(cand)
        cand.remove(drop)

        X_cand = df[cand].dropna()
        max_vif = compute_max_vif(X_cand)

        # 점수 계산: (남긴 변수 개수) - alpha * max_vif
        # alpha 를 조정해가며 '개수 우선' vs 'VIF 우선' 가중치를 바꿀 수 있음
        alpha = 0.1
        score = len(cand) - alpha * max_vif

        # 임계치 이하여서 score가 개선되면 갱신
        if max_vif <= thresh and score > best_score:
            best_score  = score
            best_subset = cand
            best_vif    = max_vif

    return best_subset, best_vif

# 사용 예시
if __name__ == "__main__":
    df = pd.read_csv('test.csv')
    # 수치형 변수만 추리기
    X = df.select_dtypes(include=[np.number]).dropna()

    best_feats, best_max_vif = random_feature_search(
        X, n_iter=2000, thresh=10.0, random_state=42
    )

    print("최적 변수 조합 (개수={}):".format(len(best_feats)), best_feats)
    print("해당 조합의 최대 VIF:", best_max_vif)


c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\junjo\